<a href="https://colab.research.google.com/github/AbhinavJain3/LinkedInsight/blob/main/LinkedInsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index-llms-azure-openai
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.2 MB/s e

In [2]:
import os
import json
import time
import requests
import numpy as np
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core.llms import ChatMessage
import re

In [3]:
def setup_llm():
    os.environ["OPENAI_API_KEY"] = "API-KEY-HERE"
    os.environ["AZURE_OPENAI_ENDPOINT"] = "AZURE_OPENAI_ENDPOINT"
    os.environ["OPENAI_API_VERSION"] = "OPENAI_API_VERSION"
    return AzureOpenAI(
        engine="gpt4o",
        model="gpt-4o",
        temperature=0.0,
        azure_endpoint="AZURE_OPENAI_ENDPOINT",
        api_key="API-KEY-HERE",
        api_version="OPENAI_API_VERSION",
    )

In [4]:
import json

def read_json_content(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data.get('content', "Content key not found.")
    except FileNotFoundError:
        return "File not found."
    except json.JSONDecodeError:
        return "Invalid JSON format."

In [6]:
articleContent = read_json_content('/content/scraped_data.json')
print(articleContent)

The founder dilemma: Should Indian startups embrace professional management?Manage SubscriptionReading ListLog Outdushyant.ips@gmail.comInvestmentsThe founder dilemma: Should Indian startups embrace professional management?Nikhil PatwardhanOctober 15, 2024 | 11 Min ReadOctober 15, 2024 | 11 Min ReadIndia’s startup ecosystem has seen many success stories, with young and ambitious founders creating disruptive businesses that upend the status quo. As these startups grow and evolve, though, a critical consideration arises—are founders best equipped to guide them through their next phase of growth?Key TakeawaysWhile founders' vision has taken major Indian startups to great heights, the need for professional management is increasingly evidentOver the years, excessive founder control has led to many startups, including Byju’s and Housing.com, facing issuesWith more startups slated to go public in coming years, various stakeholders are calling for professional management to ensure stability at

In [7]:
def clean_llm_output(llm_output):
    cleaned = llm_output.replace("```json", "").replace("```", "").strip()
    return json.loads(cleaned)

In [8]:
def generate_tags(llm, content):
    messages = [
        ChatMessage(
            role="system",
            content="""Analyze the entire content transcript provided, then assign the most relevant tags across four categories: Content Type, Tone/Audience Reaction, Structure, and Intent. For each category:
                      Content Type: Identify the primary purpose of the content by recognizing if it’s meant to inform, tell a story, share advice, inspire, celebrate an achievement, warn, or announce an update. Assign tags that match any combination of:
                      Educational Insight, Industry Analysis, Personal Storytelling, Professional Journey, Case Study, Event Recap, Thought Leadership, Inspirational Content, Advisory Content, Alert/Warning, Announcement/Update, Q&A / FAQ, Research Summary, Problem-Solution Narrative, Checklist / Step-by-Step Guide, Listicle, Celebration/Achievement, Milestone Reflection.
                      Tone and Audience Reaction: Detect the general tone, emotion, or reaction that the content seeks to evoke in the audience. Assign tags like:
                      Empowering, Reflective, Relatable, Empathetic, Encouraging, Urgent, Optimistic, Cautionary, Celebratory, Curiosity-Driven.
                      Structure: Analyze the format or layout of the content, determining if it is a narrative, instructional, or list-based structure, among others. Assign any of these tags:
                      Narrative, Comparative, Conversational, Instructional, Informative, Bullet Points / List, Data-Driven, Question-Based, Motivational, Visual-Driven, Fact-Based, Sequential.
                      Intent/Objective: Identify the overarching goal of the content. Is it to educate, engage, promote, or build community? Assign any combination of:
                      Educational, Promotional, Engagement-Oriented, Community-Building, Brand Awareness, Value-Driven, Lead Generation, Personal Branding, Feedback-Seeking, Influence-Building.
                      After analyzing each aspect, list all tags in a format categorized by Content Type, Tone/Audience Reaction, Structure, and Intent
                      Output the result in json format.
                      ```
                      {
                        {
                          "content_type": [
                              "content1",
                              "content2"
                          ],
                          "tone": [
                              "tone1",
                              "tone2"
                          ],
                          "structure": [
                              "structure1",
                              "structure2
                          ],
                          "intent": [
                              "intent1",
                              "intent2"
                          ]
                        }
                      }
                      ```
                    """
        ),
        ChatMessage(role="user",
                    content=f"""Data of the article: {content}"""),
    ]
    tags = llm.chat(messages).message.content
    return clean_llm_output(tags)

In [9]:
target_tags = generate_tags(setup_llm(), articleContent)
print(target_tags)

{'content_type': ['Industry Analysis', 'Thought Leadership', 'Case Study', 'Problem-Solution Narrative'], 'tone': ['Reflective', 'Cautionary', 'Curiosity-Driven'], 'structure': ['Informative', 'Comparative', 'Fact-Based', 'Narrative'], 'intent': ['Educational', 'Value-Driven', 'Influence-Building']}


In [15]:
import json

def get_top_texts(file_path, target_tags):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)

        scores = []

        # Calculate scores for each entry based on tag matching
        for entry in data:
            text_tags = entry['tags']
            score = sum([
                len(set(text_tags.get('content_type', [])) & set(target_tags['content_type'])),
                len(set(text_tags.get('tone', [])) & set(target_tags['tone'])),
                len(set(text_tags.get('structure', [])) & set(target_tags['structure'])),
                len(set(text_tags.get('intent', [])) & set(target_tags['intent']))
            ])
            scores.append((entry['text'], score))

        # Sort by highest score and get the top 1 entries
        top_texts = sorted(scores, key=lambda x: x[1], reverse=True)[:1]

        return [{'text': t[0], 'score': t[1]} for t in top_texts]

    except FileNotFoundError:
        return "File not found."
    except json.JSONDecodeError:
        return "Invalid JSON format."

In [16]:
result = get_top_texts('/content/output-rahul-with-tags.json', target_tags)
print(result)

[{'text': 'HSBC partnered with Visa at the start of this year to launch Zing - a competitor to Revolut & Transferwise.\n\nZing is a low cost FOREX app for UK citizens which has ~100K+ downloads so far - and it seems like this story repeats itself very often:\n\nIncumbent bank witnesses the rapid scaling of a FinTech & then launches a standalone competing product-\n\nThe typical result? The incumbent bank shuts down this project; quick history:\n\n(1) In May 2020, Santander Group launched PagoFX\n\n- Similar to Zing by HSBC, it was a separate mobile app for UK residents\n- PagoFX had committed to allowing customers to transfer money abroad with no fees for two months during COVID\n- Shut down in Sept 2021\n\n(2) In 2017, ING launched Yolt\n\n- It was a PFM (personal finance management) app in the UK\n- It had B2C & B2B services based on the UK’s Open Banking framework\n- Shut down in April 2023\n\n(3) In 2019, Royal Bank of Scotland launched Bó\n\n- This was a digital bank (Monzo compet

In [13]:
llm = setup_llm()

In [14]:
insights = [
    "Founders' vision has driven major Indian startups to success, but the need for professional management is becoming evident.",
    "Excessive founder control has led to issues in startups like Byju’s and Housing.com.",
    "With more startups going public, stakeholders are calling for professional management to ensure stability.",
    "Startups like Swiggy and BharatPe have benefited from bringing in professional management.",
    "In 2022, Swiggy hired McKinsey alum Rohit Kapoor to lead its food delivery business, making it profitable ahead of its IPO.",
    "Kapoor's success at Swiggy highlights the benefits of professional management in overcoming operational challenges.",
    "Professional management is increasingly seen as necessary for scaling startups, despite founders' initial success.",
    "The spat between Ola Electric's Bhavish Agarwal and comedian Kunal Kamra highlighted the need for professional management, as it led to a significant drop in Ola Electric’s share price.",
    "Founders often excel in innovation but may struggle with managing complex operations and meeting investor expectations as companies scale.",
    "Investors view professional management as a signal of stability and maturity, as seen in global companies like Google and Apple.",
    "Different stages of a company's growth may require different leadership styles, such as aggressive growth versus consolidation.",
    "In India, the transition to professional management is challenging due to founders' concerns about losing control and the close ties between startups and founders' identities.",
    "Founders' compelling vision often attracts investors, but excessive control can lead to issues, as seen with Byju Raveendran.",
    "Byju’s faced high-profile boardroom drama due to Raveendran's concentration of control, leading to bankruptcy proceedings.",
    "Other Indian startup founders like Rahul Yadav and Ashneer Grover have also faced issues due to excessive control and not heeding investor advice.",
    "VCs play a crucial role in guiding startups through different growth stages, akin to professional management.",
    "The need for professional management becomes urgent when startups list on stock exchanges, as seen with Ola Electric.",
    "Public spats and immature reactions by founders can have severe consequences on market perception and share prices.",
    "Professional managers are typically more controlled in their public conduct, which is crucial for listed companies.",
    "Founders should be surrounded by strong advisors, professional management, and independent directors to avoid fallout.",
    "The transition to professional management is increasingly seen as necessary for the sustainable growth of Indian startups."
]

In [17]:
def result_linkedin_post(llm, content, result):
    try:
        all_posts = []

        for entry in result:
            text = entry['text']
            posts = []

            messages = [
                ChatMessage(
                    role="system",
                    content="""
                    You are a skilled LinkedIn post creator. Using insights of an article and a Linkedin Post template, craft an engaging LinkedIn post.
                    Please follow the structure of the Template as much as possible.
                    Each paragraph should ideally contain 12-15 words, with a maximum of 20 words. Insert one blank line between each paragraph for clarity.

                    """
                ),
                ChatMessage(
                    role="user",
                    content=f"""Insights of an article:
                                {content}

                                Template of a LinkedIn Post:
                                {text}
                                """
                ),
            ]

            post = llm.chat(messages).message.content.strip()
            print(f" Posts : \n {post}\n")
            posts.append(post)

            all_posts.append(posts)

        return all_posts

    except ValueError as e:
        print(f"Error: {str(e)}")
        return None

In [18]:
result_post = result_linkedin_post(llm, insights,result)

 Posts : 
 Founders' vision has driven major Indian startups to success, but professional management is becoming essential.

Excessive founder control has led to issues in startups like Byju’s and Housing.com.

With more startups going public, stakeholders are calling for professional management to ensure stability.

Startups like Swiggy and BharatPe have benefited from bringing in professional management.

In 2022, Swiggy hired McKinsey alum Rohit Kapoor to lead its food delivery business, making it profitable ahead of its IPO.

Kapoor's success at Swiggy highlights the benefits of professional management in overcoming operational challenges.

Professional management is increasingly seen as necessary for scaling startups, despite founders' initial success.

The spat between Ola Electric's Bhavish Agarwal and comedian Kunal Kamra highlighted the need for professional management.

Founders often excel in innovation but may struggle with managing complex operations and meeting investor e